# Portfolio: `MyCoinbase`

The user portfolio is an object that is kept apart from the definition of objects related to the cryprocurrency portfolio provider (eg. Coinbase API). 

This is done for keeping a level of separation between the two layers, allowing future developement (eg. the implementation of other crypto trading services) and providing methods that are useful specifically at this level.


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from surfingcrypto import Config
from surfingcrypto.portfolio import MyCoinbase

In [3]:
parent="/Users/giorgiocaizzi/Documents/GitHub/surfingcrypto/"
c=Config(parent+"config")

## `MyCoinbase` object

The only implemented crypto trading platform is Coinbase and `MyCoinbase` is the class that under the hood of the `Portfolio` object handles the Coinbase user-profile, that is made of the client methods - inherited from the Coinbase API client wrapper object `CB` - plus some additional methods that are used for facilitating the user's sessions.

### Get active accounts

Basic `MyCoinbase` initialization offers loading all accounts that can be considered `active`, a.k.a. that have a `balance`> 0.00 EUR or USD.

In [4]:
cb=MyCoinbase(configuration=c)
print(cb)

MyCoinbase( isHistoric:False, last updated:2022-03-06 22:08:50.892283, N_accounts:6)


In [5]:
print(cb.mycoinbase_report())

AAVE : EUR 110.32
SUSHI : EUR 59.59
MATIC : EUR 348.68
UNI : EUR 106.50
ADA : EUR 305.37
USDC : EUR 0.01
---
Portfolio: EUR 930.47


### Get all accounts with transactions.

This is the `isHistoric` mode of MyCoinbase, in which the `accounts` represents all accounts that has ever had a transaction. These accounts can either be fetched all from Coinbase API or locally cached.

If the local cached account file is older than 7 days, it is automatically updated. Otherwise, accounts are loaded from the ones specified in dump file.

There is a huge difference in time of execution, loading all accounts that have transactions takes about 1m 20s versus about 12.7s to load accounts from a local dump file.

In [6]:
cb=MyCoinbase(
    active_accounts=False,
    configuration=c)
print(cb)

MyCoinbase( isHistoric:True, last updated:2022-03-06 16:08:57, N_accounts:30)


Obviously the result of `mycoinbase_report()` won't change as it ignores accounts that have `balance`==0.00, but the length of the `accounts` attribute has more accounts than when initalized for only the active ones.

In [7]:
print(cb.mycoinbase_report())

AAVE : EUR 110.32
SUSHI : EUR 59.59
MATIC : EUR 348.68
UNI : EUR 106.50
ADA : EUR 305.37
USDC : EUR 0.01
---
Portfolio: EUR 930.47


### Transaction history

Get the full transaction history via the `TransactionsHistory` class.

In [17]:
cb.get_history()

In [39]:
cb.history

TransactionsHistory(Transactions:186 - Processed:186, Unhandled:0 - Errors:9)

In [37]:
cb.history.df.head()

,type,amount,symbol,native_amount,nat_symbol,total,subtotal,total_fee,spot_price,trade_id
datetime,,,,,,,,,,
2018-10-22T06:07:05Z,fiat_deposit,100.000000,EUR,100.00,EUR,100.00,100.00,0.00,1.000000,None
2018-10-22T06:29:22Z,buy,0.017237,BTC,100.00,EUR,100.00,97.01,2.99,5627.999719,None
2018-10-22T06:29:22Z,buy,-100.000000,EUR,-100.00,EUR,NaN,NaN,NaN,1.000000,None
2018-11-09T12:05:12Z,sell,-0.017237,BTC,-93.27,EUR,93.27,96.26,2.99,5584.488743,None
2018-11-09T12:05:13Z,sell,93.270000,EUR,93.27,EUR,NaN,NaN,NaN,1.000000,None


In [38]:
cb.history.error_log[0]

{'transaction_type': 'buy',
 'account_id': 'af38031b-b9e5-5825-bb3e-77ea783ac192',
 'transaction_id': '22d9ed43-e213-5726-a127-6f80dbb8b3d1',
 'info': {'amount': -200.0, 'symbol': 'EUR', 'date': '2021-04-27T06:07:25Z'},
 'error_log': coinbase.wallet.error.NotFoundError(<Response [404]>,
                                     'not_found',
                                     'Not found',
                                     [{'id': 'not_found',
                                       'message': 'Not found'}])}